# CaCO3 structures

In [3]:
from ase.spacegroup import crystal

a =  4.99
c = 17.0615
caco3 = crystal('CaCO3', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.25), 
                          (0.2578, 0.0, 0.25)], 
                spacegroup='R -3 c', 
                cellpar=[a,a,c,90,90,120])

In [5]:
from ase.visualize import view
view(caco3)

In [17]:
from ase.io import read
caco3_cif = read("caco3_r3c.cif")
view(caco3_cif)

# NaNO3 structures

In [2]:
from ase.spacegroup import crystal
from ase.visualize import view
# T = 541K
a =  5.082
c = 17.31
nano3_low = crystal('NaNO', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.25), 
                          (0.2381, 0.0, 0.25)], 
                spacegroup='R -3 c', 
                cellpar=[a,a,c,90,90,120])

print "Low Temp: T = 541K ", nano3_low.info

# T = 563K

a =  5.084
c = 8.175
nano3_high = crystal('NaNO', [(0.0,0.0,0.0), 
                          (0.0, 0.0, 0.5), 
                          (0.2337, 0.0, 0.5)], 
                spacegroup='R -3 m', 
                cellpar=[a,a,c,90,90,120])

print "High Temp: T = 563K ", nano3_high.info

Low Temp: T = 541K  {'spacegroup': Spacegroup(167, setting=1), 'unit_cell': 'conventional'}
High Temp: T = 563K  {'spacegroup': Spacegroup(166, setting=1), 'unit_cell': 'conventional'}


In [239]:
view(nano3_low)

In [4]:
view(nano3_high)


# Create pair list

In [4]:
pairs = get_pairlist('N', 1.3, nano3_low, 'O')

In [5]:
print "NO bonds:", len(pairs)
for pair in pairs:
    print "   pair: {0:>2} {1:>2} dist: {2:04.2f}".format(pair[0], pair[1], pair[2])

NO bonds: 18
   pair:  6 14 dist: 1.21
   pair:  6 13 dist: 1.21
   pair:  6 12 dist: 1.21
   pair:  7 16 dist: 1.21
   pair:  7 17 dist: 1.21
   pair:  7 15 dist: 1.21
   pair:  8 19 dist: 1.21
   pair:  8 20 dist: 1.21
   pair:  8 18 dist: 1.21
   pair:  9 22 dist: 1.21
   pair:  9 21 dist: 1.21
   pair:  9 23 dist: 1.21
   pair: 10 26 dist: 1.21
   pair: 10 24 dist: 1.21
   pair: 10 25 dist: 1.21
   pair: 11 29 dist: 1.21
   pair: 11 27 dist: 1.21
   pair: 11 28 dist: 1.21


# Set nitrate NO bonds from 1.21 -> 1.25 using pair list

In [6]:
bond_length=1.25 
# For fix:
#   0   = fixes the 1st atom 
#   0.5 = fixes center of bond
#   1   = fixes 2nd atom

for pair in pairs:
    nano3_low.set_distance(pair[0], pair[1], bond_length, fix=0, mic=True)

In [7]:
pairs = get_pairlist('N', 1.3, nano3_low, 'O')

In [8]:
print "NO bonds:", len(pairs)
for pair in pairs:
    print "   pair: {0:>2} {1:>2} dist: {2:04.2f}".format(pair[0], pair[1], pair[2])

NO bonds: 18
   pair:  6 14 dist: 1.25
   pair:  6 13 dist: 1.25
   pair:  6 12 dist: 1.25
   pair:  7 16 dist: 1.25
   pair:  7 17 dist: 1.25
   pair:  7 15 dist: 1.25
   pair:  8 19 dist: 1.25
   pair:  8 20 dist: 1.25
   pair:  8 18 dist: 1.25
   pair:  9 22 dist: 1.25
   pair:  9 21 dist: 1.25
   pair:  9 23 dist: 1.25
   pair: 10 26 dist: 1.25
   pair: 10 24 dist: 1.25
   pair: 10 25 dist: 1.25
   pair: 11 29 dist: 1.25
   pair: 11 27 dist: 1.25
   pair: 11 28 dist: 1.25


# Create molecules from pair list

In [11]:
atom=None
molecule_ids = [None]*len(nano3_low)
molecule_counter = 0
for atomI in nano3_low:
    list1 = [pair[1] for pair in pairs if atomI.index == pair[0]]
    list2 = [pair[0] for pair in pairs if atomI.index == pair[1]]
    bonded_list = list1 + list2
    if bonded_list:
        assigned = [ molecule_ids[i] for i in bonded_list+[atomI.index]]
        
        if not any(assigned):
            for i in bonded_list+[atomI.index]:
                molecule_ids[i] = molecule_counter
            molecule_counter += 1
        else:
            molecule_id = set(uid for uid in assigned if uid is not None)
            assert(len(molecule_id) == 1)
            for i in bonded_list+[atomI.index]:
                molecule_ids[i] = list(molecule_id)[0]
    else:
        if not molecule_ids[atomI.index]:
            molecule_ids[atomI.index] = molecule_counter
        molecule_counter += 1
        
for atom in nano3_low:
    print atom.index, atom.symbol, molecule_ids[atom.index]

0 Na 0
1 Na 1
2 Na 2
3 Na 3
4 Na 4
5 Na 5
6 N 6
7 N 7
8 N 8
9 N 9
10 N 10
11 N 11
12 O 6
13 O 6
14 O 6
15 O 7
16 O 7
17 O 7
18 O 8
19 O 8
20 O 8
21 O 9
22 O 9
23 O 9
24 O 10
25 O 10
26 O 10
27 O 11
28 O 11
29 O 11


In [238]:
for atomI in nano3_low:
    print atomI.index, molecule_ids[atomI.index]

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 6
8 6
9 6
10 6
11 6
12 6
13 6
14 6
15 6
16 6
17 6
18 6
19 6
20 6
21 6
22 6
23 6
24 6
25 6
26 6
27 6
28 6
29 6


# NaNO3 MD model

In [1]:
import itertools
import numpy as np
from ase import Atoms
from ase.units import kJ, kcal, mol
from ase.spacegroup import crystal
from ase.pairlist import get_pairlist

class McDonaldNaNO3(object):
    def __init__(self, supercell=[1,1,1], cif_file=None):
        self.description = "Sodim Nitrate model for crystalline odered-disordered transition"
        self.citation = { "authors" : [ {"Last" : "Lynden-Bell", "First" : "R M"},
                                        {"Last" : "Ferrario", "First" : "M "},
                                        {"Last" : "McDonald", "First" : "I R"} ],
                          "journal" : "Journal of Physics: Condensed Matter",
                          "year" : 1989,
                          "volume" : 1,
                          "pages" : 6523-6542
                        }
        # model parameters
        a =  5.082
        c = 17.31
        self._lat_params = [a,a,c,90,90,120]
        self._no_bond_length=1.25 # N-O bond length
        
        # initialization
        self._unit_cell = None
        self._supercell = supercell
        self._mol_ids = list()
        self._molecule_counter = 1
        
        self._makeUnitCell(cif_file)
        self._makeSuperCell()
        
            
    def _makeMolecules(self,atoms,pairs):
        molecule_ids = [None]*len(atoms)
        molecule_counter = 0
        for atomI in atoms:
            list1 = [pair[1] for pair in pairs if atomI.index == pair[0]]
            list2 = [pair[0] for pair in pairs if atomI.index == pair[1]]
            bonded_list = list1 + list2
            if bonded_list:
                assigned = [ molecule_ids[i] for i in bonded_list+[atomI.index]]

                if not any(assigned):
                    for i in bonded_list+[atomI.index]:
                        molecule_ids[i] = molecule_counter
                    molecule_counter += 1
                else:
                    molecule_id = set(uid for uid in assigned if uid is not None)
                    assert(len(molecule_id) == 1)
                    for i in bonded_list+[atomI.index]:
                        molecule_ids[i] = list(molecule_id)[0]
            else:
                if not molecule_ids[atomI.index]:
                    molecule_ids[atomI.index] = molecule_counter
                molecule_counter += 1
        return molecule_ids, molecule_counter
        
    def _makeUnitCell(self,cif_file):
        if cif_file:
            atoms = read(cif_file)
        else:
            
            atoms = crystal('NaNO', [(0.0,0.0,0.0), 
                                     (0.0, 0.0, 0.25), 
                                     (0.2381, 0.0,0.25)], 
                                      spacegroup='R -3 c', 
                                      cellpar=self._lat_params)
        print "atoms cell:", atoms.cell
            
        # Get N-O pairs within 1.3 Angstroms and set bond distance to 1.25 Angstroms
        pairs = get_pairlist('N', 1.3, atoms, 'O')
        for pair in pairs:
            atoms.set_distance(pair[0], pair[1], self._no_bond_length, fix=0, mic=True)
            
        # Make molecules from atoms based on input pairs list
        mol_ids, cnt = self._makeMolecules(atoms, pairs)
        
        # Set global
        self._molecules_per_unit_cell = cnt
        self._mol_ids_unit_cell = list(mol_ids)
        self._unit_cell = atoms
        
    def _getUnitCell(self):
        if not self._unit_cell:
            self._makeUnitCell()
        return self._unit_cell.copy()
    
    def _addUnitCell(self,ix,iy,iz):
        lat = self._lat_params
        vector  = ix*self._unit_cell.cell[0]
        vector += iy*self._unit_cell.cell[1]
        vector += iz*self._unit_cell.cell[2]
        #vector = [ix*lat[0], iy*lat[1], iz*lat[2]]
        print vector
        unitCell = self._getUnitCell()
        unitCell.translate(vector)
        self.system = self.system + unitCell
        
    def _makeSuperCell(self):
        lat = self._lat_params
        sc  = self._supercell
        self.system = Atoms(cell=[sc[0]*lat[0],sc[1]*lat[1],sc[2]*lat[2],
                                  lat[3], lat[4], lat[5]], 
                            pbc=[1,1,1])
        
        ix = 0
        iy = 0
        iz = 0
        for ix in range(sc[0]):
            for iy in range(sc[1]):
                for iz in range(sc[2]):
                    self._addUnitCell(ix,iy,iz)
                    mol_ids = [  mol + self._molecule_counter for mol in self._mol_ids_unit_cell]
                    self._mol_ids += mol_ids
                    self._molecule_counter += self._molecules_per_unit_cell


            
    def lammpsParameters(self):
        # Simulation Parameters
        types = {'Na': 1, 'N': 2, 'O': 3}
        
        # Force field in units from reference (Table 1)
        #    multiples of electron charge
        charge_types = {'Na':1.00, 'N':0.95, 'O':-0.65}
        charges = [ charge_types[atom.symbol] for atom in self.system ]
        
        #      a          kJ*mol^-1     Ang^-1       kJ/mol*Ang^6
        ff = { 1 : {1 : {'A':3.1696e4, 'rho':3.155, 'B': 101}},
               2 : {2 : {'A':1.4080e5, 'rho':3.780, 'B':1084}},
               3 : {3 : {'A':2.9250e5, 'rho':4.180, 'B':1085}}
             }
        
        # Conversion to LAMMPS 'real' units
        for i in ff:
            ff[i][i]['A'] = ff[i][i]['A'] * (kJ/mol) / (kcal/mol) # units: kcal * mol^-1
            ff[i][i]['B'] = ff[i][i]['B'] * (kJ/mol) / (kcal/mol) # units: kcal * mol^-1 * Angstrom^6
            
        # Mixing to get I-J parameters
        for i in ff:
            for j in ff:
                params_ij = { 'A'   : np.sqrt(ff[i][i]['A']   * ff[j][j]['A']  ),
                              'rho' : np.sqrt(ff[i][i]['rho'] + ff[j][j]['rho']),
                              'B'   : np.sqrt(ff[i][i]['B']   * ff[j][j]['B']  )
                            }
                ff[i][j] = params_ij
                
        # Pair_coeff commands
        pair_coeff = list()
        for pair in itertools.combinations_with_replacement(ff,2):
            i = pair[0]
            j = pair[1]
            pair_coeff.append("pair_coeff {0} {1} {A:.4f} {rho:.4f} {B:.4f}".format(i, j, **ff[i][j]))
            

        # LAMMPS input
        header = ['units real',
                  'atom_style full',
                  'bond_style harmonic',
                  'kspace_style ewald/disp 1.0e-4',
                  'atom_modify map array sort 0 0']

        cmds  = ["pair_style buck/long/coul/long long long 10.0"] + pair_coeff
        cmds += ["bond_coeff * 100.0 1.565",
                 "fix 1 all nve",
                 "group NO3 type 2:3",
                 "fix 2 NO3 rigid/nve/small molecule"]

        params = { 'types' : types,
                   'charges' : charges,
                   'commands' : {'header' : header,
                                 'body'   : cmds }
                 }
        return params


In [2]:
nano3 = McDonaldNaNO3(supercell=[2,1,1])
view(nano3.system)

atoms cell: [[  5.082       0.          0.       ]
 [ -2.541       4.4011411   0.       ]
 [  0.          0.         17.31     ]]
[ 0.  0.  0.]
[ 5.082  0.     0.   ]


NameError: name 'view' is not defined

In [3]:
from ase.calculators.lammpslib import write_lammps_data
nano3 = McDonaldNaNO3(supercell=[2,1,1])
params = nano3.lammpsParameters()
system = nano3.system
mol_ids = nano3._mol_ids
write_lammps_data('config.data',
                  system,
                  params['types'],
                  cutoff=1.5,
                  molecule_ids=mol_ids,
                  charges=params['charges'],
                  bond_types=[(7,8), (8,7)],
                  angle_types=[(8,7,8)],
                  units='real')

atoms cell: [[  5.082       0.          0.       ]
 [ -2.541       4.4011411   0.       ]
 [  0.          0.         17.31     ]]
[ 0.  0.  0.]
[ 5.082  0.     0.   ]
i: 6 7 j: 13 8
i: 6 7 j: 12 8
i: 6 7 j: 44 8
i: 7 7 j: 15 8
i: 7 7 j: 16 8
i: 7 7 j: 17 8
i: 8 7 j: 18 8
i: 8 7 j: 19 8
i: 8 7 j: 20 8
i: 9 7 j: 23 8
i: 9 7 j: 22 8
i: 9 7 j: 51 8
i: 10 7 j: 24 8
i: 10 7 j: 25 8
i: 10 7 j: 26 8
i: 11 7 j: 27 8
i: 11 7 j: 28 8
i: 11 7 j: 29 8
i: 12 8 j: 6 7
i: 13 8 j: 6 7
i: 14 8 j: 36 7
i: 15 8 j: 7 7
i: 16 8 j: 7 7
i: 17 8 j: 7 7
i: 18 8 j: 8 7
i: 19 8 j: 8 7
i: 20 8 j: 8 7
i: 21 8 j: 39 7
i: 22 8 j: 9 7
i: 23 8 j: 9 7
i: 24 8 j: 10 7
i: 25 8 j: 10 7
i: 26 8 j: 10 7
i: 27 8 j: 11 7
i: 28 8 j: 11 7
i: 29 8 j: 11 7
i: 36 7 j: 42 8
i: 36 7 j: 43 8
i: 36 7 j: 14 8
i: 37 7 j: 45 8
i: 37 7 j: 46 8
i: 37 7 j: 47 8
i: 38 7 j: 48 8
i: 38 7 j: 49 8
i: 38 7 j: 50 8
i: 39 7 j: 52 8
i: 39 7 j: 53 8
i: 39 7 j: 21 8
i: 40 7 j: 54 8
i: 40 7 j: 55 8
i: 40 7 j: 56 8
i: 41 7 j: 57 8
i: 41 7 j: 58 8
i: 41 7

# PyLammps

In [4]:
#--------------------------------------------------#
# PyLammps
#--------------------------------------------------#

from lammps import IPyLammps

L = IPyLammps()

LAMMPS output is captured by PyLammps wrapper


In [5]:
for cmd in params['commands']['header']:
    L.command(cmd)
L.read_data('config.data')

[u'Reading data file ...',
 u'  triclinic box = (0 0 0) to (10.164 4.40114 17.31) with tilt (-2.541 0 0)',
 u'  1 by 1 by 1 MPI processor grid',
 u'  reading atoms ...',
 u'  60 atoms',
 u'  scanning bonds ...',
 u'  3 = max bonds/atom',
 u'  scanning angles ...',
 u'  3 = max angles/atom',
 u'  reading bonds ...',
 u'  36 bonds',
 u'  reading angles ...',
 u'  36 angles',
 u'Finding 1-2 1-3 1-4 neighbors ...',
 u'  special bond factors lj:   0          0          0         ',
 u'  special bond factors coul: 0          0          0         ',
 u'  3 = max # of 1-2 neighbors',
 u'  2 = max # of 1-3 neighbors',
 u'  2 = max # of 1-4 neighbors',
 u'  3 = max # of special neighbors']

In [6]:
for cmd in params['commands']['body']:
    print cmd

pair_style buck/long/coul/long long long 10.0
pair_coeff 1 1 7575.5258 2.5120 24.1396
pair_coeff 1 2 15966.5792 2.5084 79.0831
pair_coeff 1 3 23013.0133 2.5869 79.1196
pair_coeff 2 2 33652.0076 2.7495 259.0822
pair_coeff 2 3 48503.4451 2.6324 259.2017
pair_coeff 3 3 69909.1778 2.8914 259.3212
bond_coeff * 100.0 1.565
fix 1 all nve
group NO3 type 2:3
fix 2 NO3 rigid/nve/small molecule


In [7]:
for cmd in params['commands']['body']:
    L.command(cmd)

In [8]:
L.neighbor(10.0,'bin')

L.dump('1 all atom 1  dump.lammpstrj')

L.velocity('all create 300.0 2341')

L.command('thermo_style custom step time etotal pe ke')
L.thermo(100)

L.command('run 200')

In [ ]:
L.command('run 1000')